<a href="https://colab.research.google.com/github/sdgroeve/ML-course-VIB-2021/blob/master/notebooks/Histone_marks_dt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Histone modifications

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

random_seed = 123
np.random.seed(random_seed)

! pip install pycaret==2.2

# 1. Reading the data

In [ ]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")

In [ ]:
train_ids = train.pop("GeneId")
train_labels = train.pop("Label")

In [ ]:
test_index_col = test.pop("GeneId")

# 2. Fitting a decision tree model

The scikit-learn `DecisionTreeClassifier` class computes a decision tree predictive model from a dataset. 

To get all the options for learning you can simply type: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
help(DecisionTreeClassifier)

You notice that there are many hyperparameters to set. Some of these have a large impact on the complexity of the fitted model. An important such hyperparameter is the `max_depth` that sets a limit on how deep a decision tree can become. 

Let's create a decision tree model with `max_depth=3`:

In [ ]:
cls = DecisionTreeClassifier(max_depth=3)

This creates a decision tree model with default values for the other hyperparameters:

In [ ]:
cls

Let's create a validation set, fit the model and evaluate.

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train,train_labels,
                                                  test_size=.2, random_state=random_seed)

cls.fit(train_X,train_y)

predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)

print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)

print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,-1]),log_loss(val_y,predictions_val_prob[:,-1])))

The following code plots the fitted decision tree `cls` as a `tree.png` file:

In [ ]:
"""
from sklearn import tree
from io import StringIO
from IPython.display import Image, display
import pydotplus

out = StringIO()
tree.export_graphviz(cls, out_file=out)
graph=pydotplus.graph_from_dot_data(out.getvalue())
graph.write_png("tree.png")
"""

How do other values for for the `max_depth` hyperparameter perform?

In [ ]:
import seaborn as sns

result = []
for md in range(1,10):
  cls = DecisionTreeClassifier(max_depth=md)
  cls.fit(train_X,train_y)
  predictions_train_prob = cls.predict_proba(train_X)
  predictions_val_prob = cls.predict_proba(val_X)
  result.append([md,log_loss(train_y,predictions_train_prob[:,-1]),"train"])
  result.append([md,log_loss(val_y,predictions_val_prob[:,-1]),"val"])

toplot = pd.DataFrame(result,columns=["max_depth","log-loss","set"])
sns.lmplot(x="max_depth",y="log-loss",hue="set",data=toplot,fit_reg=False)

In [ ]:
cls = DecisionTreeClassifier(max_depth=9)

predictions_list = []
for i in range(10):
  train_X, val_X, train_y, val_y = train_test_split(train,train_labels,
                                                    test_size=.2, random_state=i)

  cls.fit(train_X,train_y)
  predictions_val = cls.predict(val_X)
  predictions_val_prob = cls.predict_proba(val_X)
  predictions_list.append(list(predictions_val_prob[:,-1]))
  print("LogLoss: %f  Accuracy: %f"%(log_loss(val_y,predictions_val_prob[:,-1]),accuracy_score(val_y,predictions_val)))

In [ ]:
tmp = pd.DataFrame(predictions_list)
predictions_avg = tmp.mean(axis=0)
print("Avg. model: %f"%(log_loss(val_y,predictions_avg)))

In [ ]:
sns.pairplot(tmp.transpose(),kind="scatter")

# 5. Ensemble learning: bagging

We have seen that bias and variance play an important role in Machine Learning. 

Let's first see what bagging can do for our dataset. 

In [ ]:
from sklearn.ensemble import BaggingClassifier

cls = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),random_state=random_seed)
                                                            
cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

With the `RandomForestClassifier` the variance of the decision tree is reduced also by selecting features for decision tree contruction at random. Let's see how far we get with default hyperparameter values.   

In [ ]:
from sklearn.ensemble import RandomForestClassifier

cls = RandomForestClassifier(random_state=random_seed)

cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

Let's get you started with PyCaret!

In [ ]:
from pycaret.classification import *

train["Label"] = train_labels
setup(data=train,target="Label",numeric_features=list(train.columns)[:-1], train_size=0.8, silent=True)


In [ ]:
from sklearn.metrics import log_loss

add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

In [ ]:
rf = lr = create_model("rf",fold=5)

In [ ]:
tune_grid = {
            "n_estimators": range(10, 300, 10),
            "max_depth": range(1, 11, 1),
            "min_impurity_decrease": [
                0,
                0.0001,
                0.001,
                0.01,
                0.0002,
                0.002,
                0.02,
                0.0005,
                0.005,
                0.05,
                0.1,
                0.2,
                0.3,
                0.4,
                0.5,
            ],
            "max_features": [1.0, "sqrt", "log2"],
            "bootstrap": [True, False],
        }

tuned_rf = tune_model(rf,tuner_verbose=2, n_iter=10, fold=5, custom_grid=tune_grid)